In [28]:
import pandas as pd
from collections import defaultdict

def deduplica_e_somma(temi):

    # Dizionario per memorizzare la somma degli ID per ciascun tema
    somma_temi = defaultdict(int)
    
    # Itera su ogni coppia tema,ID
    for coppia in temi.split(";"):
        if coppia:  # Ignora stringhe vuote
            tema, id_str = coppia.split(",")
            somma_temi[tema] += int(id_str)
    
    # Restituisce una stringa con temi unici e la somma degli ID
    return ";".join([f"{tema},{somma}" for tema, somma in somma_temi.items()])

def filtra_localita_bm(localita):
    localita_split = localita.split(";")
    localita_filtrate = [loc for loc in localita_split if "#BM#" in loc]
    return ";".join(localita_filtrate)

def process_row(row):
    v2themes = row['V2Themes']
    v2tone = float(row['V2Tone'].split(',')[0]) if pd.notna(row['V2Tone']) else 0.0  # Estrae il primo valore di V2Tone
    
    if pd.isna(v2themes):  # Se non ci sono temi, restituisci None
        return None
    
    # Dividi i temi e i pesi
    temi_pesi = []
    for theme_weight in v2themes.split(';'):
        if ',' in theme_weight:  # Verifica che il formato sia corretto
            tema, peso = theme_weight.split(',')
            temi_pesi.append((tema, int(peso)))
    
    # Filtra i temi presenti in mappatura_temi_indicatori
    temi_filtrati = [(tema, peso) for tema, peso in temi_pesi if tema in mappatura_temi_indicatori]
    
    # Calcola la somma totale dei pesi dei temi filtrati
    somma_pesi = sum(peso for _, peso in temi_filtrati)
    
    # Crea una stringa concatenata per ogni tema, indicatore e sentiment
    temi_indicatori_sentiment = []
    for tema, peso in temi_filtrati:
        weight = peso / somma_pesi if somma_pesi > 0 else 0.0
        sentiment = weight * v2tone
        indicatore = mappatura_temi_indicatori[tema]
        # Formatta la stringa: "tema|indicatore|sentiment"
        temi_indicatori_sentiment.append(f"{tema}|{indicatore}|{sentiment:.4f}")
    
    # Unisci tutte le stringhe in un'unica stringa separata da ";"
    return ";".join(temi_indicatori_sentiment) if temi_indicatori_sentiment else None


def filtra_tone(v2tone):
    return v2tone.split(',')[0]

ok_df = pd.read_csv('../Data/themes.csv')

# Crea un dizionario per mappare i temi agli indicatori
mappatura_temi_indicatori = dict(zip(ok_df['theme'], ok_df['indicator']))

# Carica il file gdelt_myanmar_GKG_2021_2024.csv
gdelt_df = pd.read_csv("/Volumes/CHERRY_SSD/gkg_myanmar_filtered.csv")

# Seleziona solo le colonne desiderate
colonne_desiderate = ['GKGRECORDID', 'DATE', 'SourceCommonName', 'V2Themes', 'V2Locations', 'V2Tone']
gdelt_df = gdelt_df[colonne_desiderate]
gdelt_df = gdelt_df.dropna()
# Applica la funzione a ogni riga
gdelt_df["V2Themes"] = gdelt_df["V2Themes"].apply(deduplica_e_somma)
# Applica la funzione a ogni riga
gdelt_df["V2Locations"] = gdelt_df["V2Locations"].apply(filtra_localita_bm)
# Applica la funzione a ogni riga del DataFrame
gdelt_df['temi_indicatori_sentiment'] = gdelt_df.apply(process_row, axis=1)

gdelt_df["V2Tone"] = gdelt_df["V2Tone"].apply(filtra_tone)

gdelt_df = gdelt_df.dropna()
desired_columns = ['GKGRECORDID', 'DATE', 'SourceCommonName', 'V2Locations', 'V2Tone', 'temi_indicatori_sentiment']
gdelt_df = gdelt_df[desired_columns]


# Mostra il risultato
gdelt_df.to_csv('../Data/gkg_data.csv', index=False)



/Volumes/CHERRY_SSD/gkg_myanmar_filtered.csv